In [0]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import h2o
from h2o.automl import H2OAutoML
import random # H2O 초기화 시 포트 번호 충돌 방지를 위해 사용
from h2o.estimators import H2OXGBoostEstimator

# 1. Spark Session 초기화 (Databricks에서는 보통 자동으로 되어 있습니다)
# spark = SparkSession.builder.appName("H2OAutoMLExample").getOrCreate()

In [0]:
# 2. CSV 데이터 불러오기
# 데이터 경로를 실제 환경에 맞게 수정해주세요.
csv_path = '/Volumes/project/default/project/UHI_data_test.csv/'
df = spark.read.option("header", "true").csv(csv_path)

print("--- 원본 Spark DataFrame 스키마 ---")
df.printSchema()
df.show(5, truncate=False)

In [0]:
# 3. 데이터 타입 변환 (df_processed 생성)
# H2O AutoML이 datetime 타입의 timestamp를 인식하도록 cast 합니다.
df_processed = df.withColumn("green_rate", col("green_rate").cast("double")) \
                  .withColumn("Building_Density", col("Building_Density").cast("double")) \
                  .withColumn("car_registration_count", col("car_registration_count").cast("long")) \
                  .withColumn("population_density", col("population_density").cast("double")) \
                  .withColumn("avg_km_per_road_km", col("avg_km_per_road_km").cast("double")) \
                  .withColumn("UHII", col("UHII").cast("double")) \
                  .withColumn("timestamp", col("timestamp").cast("timestamp")) \
                  .withColumn("suburban_temp_current", col("suburban_temp_current").cast("double"))

print("\n--- 타입 변환 후 Spark DataFrame 스키마 ---")
df_processed.printSchema()
df_processed.show(5, truncate=False)

In [0]:
# 4. Spark DataFrame을 Pandas DataFrame으로 변환
# H2O는 Pandas DataFrame 또는 H2OFrame을 입력으로 받습니다.
# 대규모 데이터셋의 경우 메모리 부족에 주의해야 합니다.
try:
    pandas_df = df_processed.toPandas()
    print(f"\nSpark DataFrame을 Pandas DataFrame으로 변환 완료. 크기: {pandas_df.shape}")
except Exception as e:
    print(f"Pandas DataFrame 변환 중 오류 발생: {e}")
    print("데이터셋이 너무 커서 메모리 부족이 발생했을 수 있습니다. 샘플링을 고려하세요.")
    # 예: pandas_df = df_processed.sample(fraction=0.1, seed=42).toPandas()

In [0]:
# 5. H2O 초기화
# Databricks에서 H2O를 초기화할 때 포트 충돌이 발생할 수 있으므로, 랜덤 포트를 사용하거나 특정 포트를 지정하는 것이 좋습니다.
# cluster_mode=True 설정은 H2O를 Spark 클러스터에서 실행합니다. (권장)
# nthreads=-1은 사용 가능한 모든 코어를 사용하겠다는 의미입니다.
try:
    h2o.init(
        # ip="localhost", # 로컬 IP 주소 지정 (선택 사항)
        port=random.randint(54321, 65535), # 랜덤 포트 사용
        nthreads=-1, # 사용 가능한 모든 스레드 사용
        min_mem_size="4G", # H2O에 할당할 최소 메모리 크기 (클러스터 환경에 맞게 조정)
        # spark_session=spark, # SparkSession 지정 (Spark 클러스터에서 H2O 실행 시)
        # strict_version_check=False # H2O 버전 체크 비활성화 (환경 문제시)
    )
    print("\nH2O Flow UI: %s" % h2o.cluster().get_web_ui())
except Exception as e:
    print(f"H2O 초기화 중 오류 발생: {e}")
    print("H2O 클러스터가 이미 실행 중이거나 포트 충돌이 발생했을 수 있습니다.")
    print("h2o.shutdown() 후 다시 시도하거나, 다른 포트 번호를 사용해보세요.")

In [0]:
# 6. Pandas DataFrame을 H2OFrame으로 변환
h2o_frame = h2o.H2OFrame(pandas_df)

print("\nH2OFrame 정보:")
h2o_frame.summary()

In [0]:
# 7. 예측 대상 (y) 및 피처 (x) 정의
y = 'UHII' # 예측하려는 컬럼
x = [col for col in h2o_frame.columns if col != y]

# H2OFrame의 컬럼 타입을 확인하는 올바른 방법은 .types 딕셔너리를 사용하는 것입니다.
timestamp_col_type = h2o_frame.types.get('timestamp')

print(f"\n'{y}'는 예측 대상입니다.")
print(f"사용될 피처들 ({len(x)}개): {x}")
print(f"H2OFrame의 'timestamp' 컬럼 타입: {timestamp_col_type}") # 올바른 타입 출력

# 이제 이 'timestamp_col_type' 변수를 사용하여 조건을 확인합니다.
if timestamp_col_type != 'time':
    print("경고: 'timestamp' 컬럼이 'time' 타입으로 인식되지 않았습니다. 변환을 다시 확인하거나 데이터 형식을 점검하세요.")
    # 만약 여전히 'time'이 아니라면, 다음 줄의 주석을 해제하여 강제로 'time' 타입으로 변환 시도
    # h2o_frame['timestamp'] = h2o_frame['timestamp'].as_time()
    # print(f"강제 변환 후 'timestamp' 컬럼 타입: {h2o_frame.types.get('timestamp')}")

In [0]:
# 8. AutoML 모델 학습 설정 및 실행
# max_runtime_secs: 최대 실행 시간 (초). 시간 부족 시 모델 수가 적을 수 있습니다.
# max_models: 최대 훈련할 모델 수.
# seed: 재현성을 위한 시드 값.
# sort_metric: 리더보드를 정렬하고 최적 모델을 선택할 때 사용할 평가 지표. (회귀: rmse, mae, rmsle, mean_residual_deviance)
# exclude_algos: 특정 알고리즘 제외 (선택 사항).
# include_algos: 특정 알고리즘만 포함 (선택 사항).
# nfolds: 교차 검증 폴드 수.
# verbosity: 상세 출력 수준 (debug, info, warn, error).

aml = H2OAutoML(
    max_runtime_secs=7200,  # 예: 1시간 (3600초) 동안 실행
    max_models=40,         # 예: 최대 20개의 모델 훈련
    seed=42,                # 재현성을 위한 시드
    sort_metric="rmse",     # 회귀 문제이므로 RMSE 사용
    # exclude_algos=["DeepLearning", "GLM"], # 특정 알고리즘 제외 예시
    # nfolds=5, # 교차 검증을 5-fold로 설정 (선택 사항, 기본값은 자동으로 설정될 수 있음)
    # verbosity="info", # 상세 정보 출력
    # project_name="UHI_Prediction" # 프로젝트 이름 지정
)

print("\n--- H2O AutoML 학습 시작 ---")
aml.train(x=x, y=y, training_frame=h2o_frame)

print("\n--- H2O AutoML 학습 완료 ---")

In [0]:
# 9. 리더보드 확인 (최고 성능 모델 목록)
leaderboard = aml.leaderboard
print("\n--- AutoML Leaderboard ---")
leaderboard.head(rows=10) # 상위 10개 모델 출력

# 10. 최고 성능 모델 확인
best_model = aml.leader
print(f"\n최고 성능 모델: {best_model.model_id}")

# 모델 성능 평가 (학습 데이터 기준)
print(f"\n--- 최고 모델 ({best_model.model_id})의 학습 데이터 기준 성능 ---")
print(f"RMSE (Root Mean Squared Error): {best_model.rmse(train=True)}")
print(f"R2 (R-squared): {best_model.r2(train=True)}")
print(f"MAE (Mean Absolute Error): {best_model.mae(train=True)}")
print(f"RMSLE (Root Mean Squared Logarithmic Error): {best_model.rmsle(train=True)}") # 필요한 경우 사용
print(f"MRD (Mean Residual Deviance): {best_model.mean_residual_deviance(train=True)}")


# 만약 검증 데이터를 분리했다면:
# print(f"최고 모델 ({best_model.model_id})의 검증 데이터 RMSE: {best_model.rmse(valid=True)}")

In [0]:
# 피쳐 중요도
# (이전 코드에서 H2OAutoML 학습 완료 후 best_model 객체가 있다고 가정)
# best_model = aml.leader

# 피처 중요도 테이블 가져오기
varimp_df = best_model.varimp(use_pandas=True)

print("\n--- 최고 모델의 피처 중요도 (상위 10개) ---")
# Importance, Percentage, Cumulative Sum 컬럼이 포함됩니다.
print(varimp_df.head(10))

# 전체 피처 중요도를 보고 싶다면
# print(varimp_df)

# 중요도 순으로 정렬되어 있으므로, 처음 몇 개만 봐도 주요 피처를 알 수 있습니다.

In [0]:
# 11. 예측 수행 (새로운 데이터 또는 기존 데이터로 테스트)
# 테스트 데이터셋이 있다고 가정하고, 동일한 전처리 (type casting)만 수행한 후 사용합니다.
# 예를 들어, test_df_processed = spark.read.option("header", "true").csv('/path/to/test_data.csv')...
# test_pandas_df = test_df_processed.toPandas()
# test_h2o_frame = h2o.H2OFrame(test_pandas_df)

# 예시: 학습 데이터를 다시 사용하여 예측 (실제로는 이렇게 하지 않습니다)
predictions = best_model.predict(h2o_frame)
print("\n--- 예측 결과 (일부) ---")
predictions.head(5)

# 예측 결과를 Pandas DataFrame으로 변환
predictions_pandas = predictions.as_data_frame()
print("\n--- 예측 결과 Pandas DataFrame (일부) ---")
print(predictions_pandas.head())

In [0]:
save_path = h2o.save_model(model=best_model, path="/tmp/mymodel", force=True)

In [0]:
best_model = h2o.load_model(save_path)

In [0]:
new_data_spark_df_raw = spark.createDataFrame([
    # 예측 데이터1
    ('Jungnang-gu', 0.364, 1.178, 113569, 21597.946, 8848.239521, '2020-08-01T18:00:00.000Z', 20.79),
    # 예측 데이터2
    ('Dobong-gu', 0.569, 0.764, 96298, 15852.833, 5277.944143, '2020-08-01T18:00:00.000Z', 20.79),
    # 예측 데이터3
    ('Jung-gu', 0.255, 2.07, 53324, 13174.096, 14927.25044, '2020-08-01T18:00:00.000Z', 20.79),
    # 예측 데이터4
    ('Jung-gu', 0.1, 2.07, 53324, 13174.096, 14927.25044, '2020-08-01T18:00:00.000Z', 20.79),
    # 예측 데이터5
    ('Jung-gu', 0.7, 0.6, 53324, 13174.096, 14927.25044, '2020-08-01T18:00:00.000Z', 20.79),
    # 예측 데이터6
    ('Gangnam-gu', 0.7, 0.6, 53324, 13174.096, 14927.25044, '2020-08-01T18:00:00.000Z', 20.79),
    # 예측 데이터7
    ('Jung-gu', 0.255, 2.07, 53324, 13174.096, 14927.25044, '2024-07-12T15:00:00.000Z', 25.21),
    # 예측 데이터8
    ('Jung-gu', 0.255, 2.07, 53324, 13174.096, 14927.25044, '2024-07-17T03:00:00.000Z', 21.17),
    # 예측 데이터9
    ('Gangbuk-gu', 0.628, 0.555, 76105, 13202.076, 5240.425625, '2025-08-19T16:00:00.000Z', 28.9),
    # 예측 데이터10
    ('Gangnam-gu', 0.628, 0.555, 76105, 13202.076, 5240.425625, '2025-08-19T16:00:00.000Z', 28.9)
], schema=[
    'District', 'green_rate', 'Building_Density', 'car_registration_count',
    'population_density', 'avg_km_per_road_km', 'timestamp', 'suburban_temp_current'
])

# 2. 학습 데이터와 동일한 Spark DataFrame 전처리 적용
# 여기서 timestamp 컬럼을 Spark의 timestamp 타입으로 캐스팅합니다.
new_data_spark_df_processed = new_data_spark_df_raw.withColumn("green_rate", col("green_rate").cast("double")) \
                                                     .withColumn("Building_Density", col("Building_Density").cast("double")) \
                                                     .withColumn("car_registration_count", col("car_registration_count").cast("long")) \
                                                     .withColumn("population_density", col("population_density").cast("double")) \
                                                     .withColumn("avg_km_per_road_km", col("avg_km_per_road_km").cast("double")) \
                                                     .withColumn("timestamp", col("timestamp").cast("timestamp")) \
                                                     .withColumn("suburban_temp_current", col("suburban_temp_current").cast("double"))

print("\n--- 예측할 새로운 Spark DataFrame (전처리 후) ---")
new_data_spark_df_processed.show(truncate=False)
new_data_spark_df_processed.printSchema()


# 3. Spark DataFrame을 Pandas DataFrame으로 변환
new_data_pandas_df = new_data_spark_df_processed.toPandas()

# Pandas에서 datetime 타입으로 이미 잘 변환되었는지 확인
print("\n--- 예측할 새로운 Pandas DataFrame ---")
display(new_data_pandas_df)
print("\nTimestamp 컬럼 타입 (Pandas):", new_data_pandas_df['timestamp'].dtype)


# 4. Pandas DataFrame을 H2OFrame으로 변환 (예측 데이터)
# 학습 시와 동일한 방식으로 H2O가 'timestamp' 컬럼을 'time' 타입으로 자동 추론하기를 기대합니다.
new_h2o_frame = h2o.H2OFrame(new_data_pandas_df)


# 5. H2OFrame의 'timestamp' 컬럼 타입 확인
# H2O가 'time' 타입으로 잘 인식했는지 최종 확인합니다.
h2o_frame_timestamp_type = new_h2o_frame.types.get('timestamp')
print(f"\n변환 후 예측 데이터 'timestamp' 컬럼 H2O 타입: {h2o_frame_timestamp_type}")

if h2o_frame_timestamp_type != 'time':
    print("경고: 'timestamp' 컬럼이 'time' 타입으로 인식되지 않았습니다. 모델 예측에 문제가 발생할 수 있습니다.")


print("\n--- 예측할 새로운 H2OFrame 정보 ---")
new_h2o_frame.show_summary()


# 6. 최고 성능 모델로 예측 수행
predictions = best_model.predict(new_h2o_frame)

# 7. 예측 결과를 Pandas DataFrame으로 변환하여 원본 데이터와 함께 보기
predictions_df = predictions.as_data_frame()
result_df = pd.concat([new_data_pandas_df, predictions_df], axis=1)


print("\n--- 입력 데이터와 최종 UHII 예측 결과 ---")
display(result_df)